In [1]:
#Load Libraries
import json
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
# team = 'Arsenal'

In [3]:
# data_dir = f"D:\Personal\Football Analysis\Event Data\Team Analysis\{team}\{team}_stats\data"

In [4]:
# if not os.path.exists(data_dir):
#     os.mkdir(data_dir)

In [5]:
os.chdir(r'D:\Personal\Football Analysis\Event Data\Team Analysis\EPL_23_24\data')

In [6]:
def scrape_xGbuildup(scrape_url):
    page_connect = urlopen(scrape_url)
    page_html = BeautifulSoup(page_connect,"html.parser")
    json_Raw_string = page_html.findAll(name='script')[3].string
    json_Raw_string
    start_ind = json_Raw_string.index('\\')
    stop_ind = json_Raw_string.index("')")
    json_data = json_Raw_string[start_ind:stop_ind]
    json_data=json_data.encode("utf8").decode("unicode_escape")
    data = json.loads(json_data)
    final_json = pd.json_normalize(data)
    final_json=final_json.apply(pd.to_numeric, errors="ignore")
    final_json_player = final_json.copy()
    final_json_player = final_json_player[['time','player_name','team_title', 'xGChain', 'xGBuildup']]
    final_df = final_json[['team_title', 'xGChain', 'xGBuildup']]
    #selecting certain rows which has , in it in the team column
    remove = final_df[final_df['team_title'].str.contains(',')].index
    #removing those rows
    final_df.drop(remove, inplace = True)
    final = final_df.groupby(by=["team_title"]).sum().reset_index()
    #os.chdir(data_dir)
    final.to_csv('xgbuildup.csv')
    final_json_player.to_csv('xG Buildup player.csv')

In [7]:
scrape_url = "https://understat.com/league/EPL"
scrape_xGbuildup(scrape_url)